In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('/kaggle/input/insurance/insurance.csv')

In [ ]:
df.info() #to check if there's null values

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
sns.histplot(df['charges'], kde=True)

In [ ]:
sns.pairplot(df)
plt.show()

In [ ]:
# One-Hot Encoding for Categorical Variables
Cate_cols = ['sex', 'smoker', 'region']
df = pd.get_dummies(df, columns= Cate_cols, drop_first=True)

In [ ]:
#defining deatures and targets 
X = df.drop('charges',axis=1)

In [ ]:
y=df['charges']

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [ ]:
#Scaling features
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# **Linear Regression**

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train,y_train)

In [ ]:
y_pred= model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error 
rmse_linear = np.sqrt(mean_squared_error(y_test,y_pred))

In [ ]:
rmse_linear

#### Calculate and plot residuals

In [ ]:
test_residuals = y_test - y_pred

plt.figure(figsize=(8,6))
sns.scatterplot(x=y_test, y=test_residuals, color='purple', alpha=0.6)
plt.axhline(y=0, color='red', linestyle='--', linewidth=2)

plt.title("Residual Plot")
plt.xlabel("Actual Charges")
plt.ylabel("Residuals (Actual - Predicted)")
plt.grid(True)
plt.show()

#### Conclusion
*The residual plot shows that our linear regression model is not predicting well for people with very high medical charges. The difference between the actual and predicted values becomes bigger as the charges increase. This means the model is not able to capture some patterns in the data, especially for higher values. So, a more advanced model like polynomial regression or decision trees might work better.*

# **Polynomial Regression**

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

### Checking best degree 

In [ ]:
X.head()

In [ ]:
# Since X is already scaled 
train_rmse_error = []
test_rmse_error = []

In [ ]:
for d in range(1,10):
    polynomial_converter = PolynomialFeatures(degree=d,include_bias = False)
    poly_feature = polynomial_converter.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(poly_feature, y, test_size=0.3, random_state=101)
    
    model = LinearRegression()
    model.fit(X_train , y_train)

    train_predi = model.predict(X_train)
    test_predi = model.predict(X_test)

    train_rmse = np.sqrt(mean_squared_error(y_train,train_predi))
    test_rmse = np.sqrt(mean_squared_error(y_test,test_predi))

    train_rmse_error.append(train_rmse)
    test_rmse_error.append(test_rmse)

### Visualize

In [ ]:
plt.plot(train_rmse_error,label='TRAIN RMSE') 
plt.plot(test_rmse_error,label='TEST RMSE')

plt.ylabel('RMSE')
plt.xlabel('Degree of Poly')
plt.legend()      #plt.legend() tells matplotlib to show a box with those labels

In [ ]:
test_rmse_error

In [ ]:
train_rmse_error

In [ ]:
# Create DataFrame
rmse_df = pd.DataFrame({
    'Degree': list(range(1, 10)),
    'Train_RMSE': train_rmse_error,
    'Test_RMSE': test_rmse_error
})

# Display
rmse_df

* Degree 2 provides the best fit.
* It achieves a low test RMSE without overfitting, unlike higher degrees which reduce training error but significantly increase testing error, indicating overfitting.

#### **Conclusion:** *Polynomial Regression (Degree 2) performed better than Linear Regression (Degree 1 by achieving a lower RMSE on the test set.*

# K-Nearest Neighbors (KNN) 

In [ ]:
df.head()

In [ ]:
#defining deatures and targets 
X = df.drop('charges',axis=1)

In [ ]:
y=df['charges']

In [ ]:
X.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [ ]:
#Scaling features
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

### Choosing the best K
* KNeighborsClassifier → for classification tasks
* KNeighborsRegressor → for regression tasks

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

test_rmse_errors = []

for k in range(1, 30):
    knn_model = KNeighborsRegressor(n_neighbors=k)
    knn_model.fit(scaled_X_train, y_train)
    y_pred_test = knn_model.predict(scaled_X_test)
    
    rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    test_rmse_errors.append(rmse)

### Plot Error vs. K

In [ ]:
plt.figure(figsize=(10,6),dpi=200)
plt.plot(range(1,30),test_rmse_errors,label='Test Error')
plt.legend()
plt.ylabel('Error Rate')
plt.xlabel("K Value")

In [ ]:
test_rmse_errors

Once you have a good idea of which range of k works best (say, 5–10), now you:
* Automate finding the best k.
* Use cross-validation for more reliable evaluation.
* Combine scaling + modeling in one unit (Pipeline).

### Pipeline + GridSearchCV for KNN Regression

In [ ]:
scaler = StandardScaler()
knn = KNeighborsRegressor()

operations = [('scaler',scaler),('knn',knn)]
from sklearn.pipeline import Pipeline
pipe = Pipeline(operations)

In [ ]:
from sklearn.model_selection import GridSearchCV

k_values = list(range(1, 20))
param_grid = {'knn__n_neighbors': k_values}

grid_model = GridSearchCV(pipe, param_grid, cv=5, scoring='neg_root_mean_squared_error')
grid_model.fit(X_train, y_train)

### 1. Check the Best Parameters Found by GridSearch

In [ ]:
print("Best k value:", grid_model.best_params_)

### View the Best Score (lowest RMSE)

In [ ]:
print("Best CV RMSE:", -grid_model.best_score_)

### Make Predictions on Test Set

In [ ]:
y_pred = grid_model.predict(X_test)

### Evaluate Model Performance on Test Set

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

knn_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
knn_r2 = r2_score(y_test, y_pred)

print("Test RMSE:", knn_rmse)
print("Test R² Score:", knn_r2)

### Calculate and plot Residuals

In [ ]:
residuals = y_test - y_pred
plt.figure(figsize=(10,6), dpi=100)
plt.scatter(y_test, residuals, color='purple', alpha=0.6)
plt.axhline(y=0, color='red', linestyle='--')
plt.xlabel("Actual Values")
plt.ylabel("Residuals (Actual - Predicted)")
plt.title("Residuals vs Actual Values")
plt.grid(True)
plt.show()

#### **Conclusion**: Your model is good at predicting small numbers, but it's very bad at predicting large numbers. The bigger the true value is, the bigger the potential error of your model's prediction will be.